In [ ]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
import pandas as pd
import soundfile as sf
import numpy as np
from sklearn.utils import shuffle
from sys import platform
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
seed=42
n_class="6class"

class_dist = {0:1000,1:1000,2:877,3:555,4:1000,5:1000}

In [ ]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']
d=np.load(f'./resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)
lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')

feature_df=df[['label','audio','band_name']]
feature_df.groupby('label')['audio'].count()

In [ ]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']
d=np.load(f'./resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)
lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')

feature_df=df[['label','audio','band_name','video_id']]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(3,'label_mapped',mapping)
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['audio','band_name','video_id']].to_numpy()
y=le.fit_transform(feature_df['label_mapped'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

video_ids=X_under[:,2]
band_names = X_under[:,1]
# X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

# train_video_ids=video_ids[train]
# test_video_ids=video_ids[test]

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

train_bands = X_train[:,1]
test_bands = X_test[:,1]
valid_bands = X_valid[:,1]

train_songs = X_train[:,2]
test_songs = X_test[:,2]
valid_songs = X_valid[:,2]

X_train = X_train[:,0]
X_test = X_test[:,0]
X_valid = X_valid[:,0]

d=pd.DataFrame()
d['y_train'] = y_train
d['blah'] = 1
print('TRAIN')
print(d.groupby('y_train')['blah'].count())
train = d['blah'].sum()

d=pd.DataFrame()
d['y_test'] = y_test
d['blah'] = 1
print('TEST')
print(d.groupby('y_test')['blah'].count())
test = d['blah'].sum()
d=pd.DataFrame()
d['y_valid'] = y_valid
d['blah'] = 1
print('VALID')
print(d.groupby('y_valid')['blah'].count())
valid = d['blah'].sum()

print(f"Train:Test:Validation - {train}:{test}:{valid}")


y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-rawaudio.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-rawaudio.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-rawaudio.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-rawaudio.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-rawaudio.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-rawaudio.npy', y_valid)

In [ ]:
le.classes_

In [ ]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']
d=np.load(f'./resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)
lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')

feature_df=df[['label','audio','band_name','video_id']]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['audio','band_name','video_id']].to_numpy()
y=le.fit_transform(feature_df['label'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
# X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

d=pd.DataFrame()
d['y_train'] = y_train
d['blah'] = 1
print('TRAIN')
print(d.groupby('y_train')['blah'].count())
train = d['blah'].sum()

d=pd.DataFrame()
d['y_test'] = y_test
d['blah'] = 1
print('TEST')
print(d.groupby('y_test')['blah'].count())
test = d['blah'].sum()
d=pd.DataFrame()
d['y_valid'] = y_valid
d['blah'] = 1
print('VALID')
print(d.groupby('y_valid')['blah'].count())
valid = d['blah'].sum()

print(f"Train:Test:Validation - {train}:{test}:{valid}")

train_bands = X_train[:,1]
test_bands = X_test[:,1]
valid_bands = X_valid[:,1]

train_songs = X_train[:,2]
test_songs = X_test[:,2]
valid_songs = X_valid[:,2]

X_train = X_train[:,0]
X_test = X_test[:,0]
X_valid = X_valid[:,0]

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-rawaudio.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-rawaudio.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-rawaudio.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-rawaudio.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-rawaudio.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-rawaudio.npy', y_valid)

# VGGish

In [ ]:
d=np.load(f'./resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)

feature_df=df[['label','vggish','band_name']]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['vggish','band_name']].to_numpy()
y=le.fit_transform(feature_df['label'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
X_under=X_under[:,0]#.reshape(-1,1).flatten()
X_under=np.concatenate(X_under).reshape(X_under.shape[0],128)

y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-vggish.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-vggish.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-vggish.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-vggish.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-vggish.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-vggish.npy', y_valid)

# Features

In [ ]:
cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']

d=np.load(f'./resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')
df

feature_df=df[['label', 'band_name', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std']]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','band_name']].to_numpy()
y=le.fit_transform(feature_df['label'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,-1]
X_under=X_under[:,:-1]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-features_unnormalized.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-features_unnormalized.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-features_unnormalized.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-features_unnormalized.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-features_unnormalized.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-features_unnormalized.npy', y_valid)

# MFCCs only

In [ ]:
cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']

d=np.load(f'./resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')
df

feature_df=df[['label', 'band_name', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std']]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std','band_name']].to_numpy()
y=le.fit_transform(feature_df['label'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,-1]
X_under=X_under[:,:-1]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-mfcc_only_unnormalized.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-mfcc_only_unnormalized.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-mfcc_only_unnormalized.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-mfcc_only_unnormalized.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-mfcc_only_unnormalized.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-mfcc_only_unnormalized.npy', y_valid)

# Features ONLY

In [ ]:
cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']

d=np.load(f'./resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'../../dataset/lookup.csv')

df=df.merge(lut[['video_id','band_name']],on='video_id')
df

feature_df=df[['label', 'band_name', 'average_zcr',
       'zcr_stddev','centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std']]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=class_dist,random_state=seed)
X = feature_df[['average_zcr',
       'zcr_stddev','centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','band_name']].to_numpy()
y=le.fit_transform(feature_df['label'].to_numpy())
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,-1]
X_under=X_under[:,:-1]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=seed)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=seed)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot, random_state=seed)
X_test,y_test=shuffle(X_test,y_test_hot, random_state=seed)
X_valid,y_valid=shuffle(X_valid,y_valid_hot, random_state=seed)

np.save(f'./resources/working_data/{n_class}_x_train-features_only_unnormalized.npy', X_train)
np.save(f'./resources/working_data/{n_class}_x_test-features_only_unnormalized.npy', X_test)
np.save(f'./resources/working_data/{n_class}_x_valid-features_only_unnormalized.npy', X_valid)

np.save(f'./resources/working_data/{n_class}_y_train-features_only_unnormalized.npy', y_train)
np.save(f'./resources/working_data/{n_class}_y_test-features_only_unnormalized.npy', y_test)
np.save(f'./resources/working_data/{n_class}_y_valid-features_only_unnormalized.npy', y_valid)